In [1]:
import transformers
import torch
import accelerate

/Users/wiktormorawski/Work/kafkAI/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
model_id_local = "./models/Meta-Llama-3-8B"

In [34]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto", 
)
pipeline("Hey how are you doing today?")

## Error 

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 137518.16it/s]


ValueError: Could not load model meta-llama/Meta-Llama-3-8B with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/Users/wiktormorawski/Work/kafkAI/.venv/lib/python3.11/site-packages/transformers/pipelines/base.py", line 283, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wiktormorawski/Work/kafkAI/.venv/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py", line 563, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wiktormorawski/Work/kafkAI/.venv/lib/python3.11/site-packages/transformers/modeling_utils.py", line 3735, in from_pretrained
    disp

In [35]:
import ollama
ollama.list()

{'models': [{'name': 'llama2:latest',
   'model': 'llama2:latest',
   'modified_at': '2024-04-21T16:26:32.151220372+02:00',
   'size': 3826793677,
   'digest': '78e26419b4469263f75331927a00a0284ef6544c1975b826b15abdaef17bb962',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '7B',
    'quantization_level': 'Q4_0'}}]}

In [41]:
import ollama
response = ollama.chat(model='llama2', messages=[
  {
    'role': 'user',
    'content': 'What is the maximum number of charaters you can generate?',
  },
], )
print(response['message']['content'])


I'm just an AI, I don't have a specific limit on the number of characters I can generate. However, the length of the generated text will depend on the complexity and length of the input prompts, as well as the model's capacity to process and generate text.

In general, the longer the input prompt or the more complex the task, the longer and more detailed the generated text will be. I can generate text of varying lengths, from a few words to several sentences or even entire paragraphs.

If you have any specific requirements or preferences for the length of the generated text, feel free to let me know and I'll do my best to accommodate them!


In [ ]:
#save_response_to_pdf

#create_parametrization_of_generation



In [95]:
from prompts.prompts import MOST_BASIC_PROMPT

In [64]:
OUTPUT_PATH = "./outputs/"
import os
from fpdf import FPDF

def save_to_pdf(text, filename):
    pdf = FPDF()
    pdf.set_margins(10, 10)
    pdf.add_page()
    pdf.set_font("Arial", size = 12)
    pdf.multi_cell(200, 10, txt = text, align = 'L', )
    pdf.output(os.path.join(OUTPUT_PATH, filename))



In [78]:
def get_single_ai_response_and_save_it(model_id, message):
  """This function generates response from a signle user prompt and saves it to a PDF file."""
  # Get response
  response = ollama.chat(model=model_id, messages=[
    {
      'role': 'user',
      'content': message['prompt'],
    },
  ], )

  # Save response to PDF
  text = f'model: {model_id}\n created_at: {response["created_at"]}\n prompt: {message["prompt"]}\n\n response: \n {response["message"]["content"]}' 
  
  filename = f'{message["name"]}_{model_id}_{response["created_at"]}.pdf' 
  
  save_to_pdf(text, filename)

In [96]:
get_single_ai_response_and_save_it('llama2', MOST_BASIC_PROMPT)

In [79]:
x = get_single_ai_response_and_save_it('llama2', {'name': 'test_prompt', 'prompt': "what's your name?"})